In [162]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm

In [331]:
import pickle

In [276]:
df_2018_03 = pd.read_csv('../ravenpack/RavenPackAnalytics_AllEntities_1.0_2018/2018-03.csv', low_memory=False)
df_2019_03 = pd.read_csv('../ravenpack/RavenPackAnalytics_AllEntities_1.0_2019/2019-03.csv', low_memory=False)

In [277]:
df_2020_0103 = pd.read_csv('2020_JanMar.csv', low_memory=False)

In [278]:
df_2020_0103['TIMESTAMP_UTC'] = pd.to_datetime(df_2020_0103['TIMESTAMP_UTC'])

In [279]:
df_2020_03 = df_2020_0103[df_2020_0103.TIMESTAMP_UTC>'2020-03-01']

In [280]:
df_2018_03['TIMESTAMP_UTC'] = pd.to_datetime(df_2018_03['TIMESTAMP_UTC'])

In [281]:
df_2019_03['TIMESTAMP_UTC'] = pd.to_datetime(df_2019_03['TIMESTAMP_UTC'])

In [282]:
# df_2018_03[df_2018_03.TIMESTAMP_UTC.dt.strftime('%Y-%m-%d')=='2018-03-15']

In [283]:
df_2018_03 = df_2018_03[df_2018_03.ENTITY_TYPE.isin(['COMP'])]

In [284]:
df_2019_03 = df_2019_03[df_2019_03.ENTITY_TYPE.isin(['COMP'])]

In [285]:
df_2020_03 = df_2020_03[df_2020_03.ENTITY_TYPE.isin(['COMP'])]

In [286]:
df_2018_03 = df_2018_03[df_2018_03.TOPIC=='business']
df_2019_03 = df_2019_03[df_2019_03.TOPIC=='business']
df_2020_03 = df_2020_03[df_2020_03.TOPIC=='business']

In [287]:
df_2018_03.dropna(subset=['TYPE'], inplace=True)
df_2019_03.dropna(subset=['TYPE'], inplace=True)
df_2020_03.dropna(subset=['TYPE'], inplace=True)

In [301]:
df_2018_03 = df_2018_03[['TIMESTAMP_UTC', 'RP_STORY_ID', 'RP_ENTITY_ID', 'ENTITY_TYPE',
       'ENTITY_NAME', 'EVENT_SIMILARITY_KEY',
       'TOPIC', 'GROUP', 'TYPE', 'SUB_TYPE', 'RELATED_ENTITY', 'CATEGORY',
       'EVENT_TEXT', 'NEWS_TYPE', 'RP_SOURCE_ID', 'SOURCE_NAME',
       
       'RP_STORY_EVENT_INDEX', 'RP_STORY_EVENT_COUNT', 
       'PROVIDER_ID', 'PROVIDER_STORY_ID', 'HEADLINE']]

In [303]:
df_2019_03 = df_2019_03[['TIMESTAMP_UTC', 'RP_STORY_ID', 'RP_ENTITY_ID', 'ENTITY_TYPE',
       'ENTITY_NAME', 'EVENT_SIMILARITY_KEY',
       'TOPIC', 'GROUP', 'TYPE', 'SUB_TYPE', 'RELATED_ENTITY', 'CATEGORY',
       'EVENT_TEXT', 'NEWS_TYPE', 'RP_SOURCE_ID', 'SOURCE_NAME',
       
       'RP_STORY_EVENT_INDEX', 'RP_STORY_EVENT_COUNT', 
       'PROVIDER_ID', 'PROVIDER_STORY_ID', 'HEADLINE']]

In [304]:
df_2020_03 = df_2020_03[['TIMESTAMP_UTC', 'RP_STORY_ID', 'RP_ENTITY_ID', 'ENTITY_TYPE',
       'ENTITY_NAME', 'EVENT_SIMILARITY_KEY',
       'TOPIC', 'GROUP', 'TYPE', 'SUB_TYPE', 'RELATED_ENTITY', 'CATEGORY',
       'EVENT_TEXT', 'NEWS_TYPE', 'RP_SOURCE_ID', 'SOURCE_NAME',
       
       'RP_STORY_EVENT_INDEX', 'RP_STORY_EVENT_COUNT', 
       'PROVIDER_ID', 'PROVIDER_STORY_ID', 'HEADLINE']]

In [305]:
df_2018_03.to_csv('2018marchfilteredIcompIbusinessTyesTYPE.csv')

In [306]:
df_2019_03.to_csv('2019marchfilteredIcompIbusinessTyesTYPE.csv')

In [307]:
df_2020_03.to_csv('2020marchfilteredIcompIbusinessTyesTYPE.csv')

In [326]:
# create event sequences
def createsequence(companyid,df):
    companydf = df[df.RP_ENTITY_ID==companyid]
    
    keynodf = companydf[companydf.EVENT_SIMILARITY_KEY.isna()]
    keyyesdf = companydf[companydf.EVENT_SIMILARITY_KEY.notna()]
    keyyesdf.drop_duplicates(subset=['EVENT_SIMILARITY_KEY'], keep="first", inplace=True)
    requireddf = pd.concat([keynodf,keyyesdf])
    requireddf.sort_values(by=['TIMESTAMP_UTC'],ascending=True, inplace=True)
    
    sequence = []
    sequence = list(requireddf.TYPE)
    
    return sequence

In [327]:
dfbigdict = {}
# companyids = []
# companyids = list(df_2018_03.RP_ENTITY_ID.unique())
# dfcompdict = {}

In [328]:
# for company in tqdm(companyids):
#     dfcompdict[company] = createsequence(company,df_2018_03)

In [330]:
companyids2018 = []
companyids2018 = list(df_2018_03.RP_ENTITY_ID.unique())
dfcompdict2018 = {}
for company in tqdm(companyids2018):
    dfcompdict2018[company] = createsequence(company,df_2018_03)
dfbigdict['2018'] = dfcompdict2018


100%|██████████| 18627/18627 [51:08<00:00,  6.07it/s]


NameError: name 'pickle' is not defined

In [336]:
pickle.dump(dfcompdict2018, open( "save2018.pickle", "wb" ) )

In [337]:
companyids2019 = []
dfcompdict2019 = {}
companyids2019 = list(df_2019_03.RP_ENTITY_ID.unique())
for company in tqdm(companyids2019):
    dfcompdict2019[company] = createsequence(company,df_2019_03)
dfbigdict['2019'] = dfcompdict2019
with open('dfcompdict2019.pickle', 'wb') as filehandle:
    pickle.dump(dfcompdict2019, filehandle)

100%|██████████| 19988/19988 [27:11<00:00, 12.25it/s]


In [338]:
companyids2020 = []
dfcompdict2020 = {}
companyids2020 = list(df_2020_03.RP_ENTITY_ID.unique())
for company in tqdm(companyids2020):
    dfcompdict2020[company] = createsequence(company,df_2020_03)
dfbigdict['2020'] = dfcompdict2020
with open('dfcompdict2020.pickle', 'wb') as filehandle:
    pickle.dump(dfcompdict2020, filehandle)

100%|██████████| 21241/21241 [36:51<00:00,  9.60it/s]


In [341]:
# 2018 edges from sequence
edgedict2018 = {}
for key in tqdm(dfcompdict2018):
    templist = []
    for i in range(len(dfcompdict2018[key])):
        if i == len(dfcompdict2018[key])-1:
            continue
        else:
            templist.append([dfcompdict2018[key][i],dfcompdict2018[key][i+1]])
    edgedict2018[key] = templist

for key in edgedict2018:
    if not edgedict2018[key]:
        edgedict2018.pop(key)

100%|██████████| 18627/18627 [00:04<00:00, 4521.96it/s]


In [358]:
for key in companyids2018:
    if not edgedict2018[key]:
        edgedict2018.pop(key)

KeyError: 'AC097E'

In [ ]:
#adjacency matrix for one company in one year march
# sequence A T G C C G T A G C T A T G C A
list = []
sequencelist = ['A','T','G','C','A','G','T','A','C']
for i in range(len(sequencelist)):
    if i == len(sequencelist)-1:
        continue
    else:
        list.append(sequencelist[i],sequencelist[i+1])